In [21]:
import numpy as np
import pandas as pd
from sklearn import preprocessing
from keras.layers import Input, Dense
from keras.models import Model
from keras.callbacks import TensorBoard

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 30)
pd.set_option('precision', 7)
pd.options.display.float_format = '{:,.3f}'.format
import warnings
warnings.simplefilter(action = "ignore", category = FutureWarning)

'''
读入一支股票指定年份的ohlcv数据
输入:baseDir,stockCode为字符, startYear,yearNum为整数，
输出:dataframe
'''
def readWSDFile(baseDir, stockCode, startYear, yearNum=1):
    # 解析日期
    filename = baseDir+stockCode+'/'+stockCode+'.csv'
    print (filename, "===============")
    dateparse = lambda x: pd.datetime.strptime(x, '%Y/%m/%d').date()
    df = pd.read_csv(filename, index_col=0, sep=',', header=None,
                            skiprows=1, names=['Date','Pre_Close','Open','High','Low','Close','Chg','Chg_Range',
                                               'Volume','Amount','Turn'],
                           parse_dates=True, date_parser=dateparse)
    return df['2013-01-04':'2015-12-31']

'''
读入一支股票指定年份的技术指标
输入:baseDir,stockCode为字符, startYear,yearNum为整数，
输出:dataframe
'''
def readWSDIndexFile(baseDir, stockCode, startYear, yearNum=1):
    # 解析日期
    dateparse = lambda x: pd.datetime.strptime(x, '%Y/%m/%d').date()

    df = 0
    for i in range(yearNum):
        tempDF = pd.read_csv(baseDir+'I'+stockCode+'/wsd_'+stockCode+'_'+str(startYear+i)+'.csv', index_col=0, sep=',', parse_dates=True, date_parser=dateparse
                             # , usecols=usecols
                             )
        if i==0: df = tempDF
        else: df = df.append(tempDF)
    return df


# prepare data
baseDir = '../'
stockCodes = ['000300.SH']
i = 0
startYear = 2013
number = 3
df = readWSDFile(baseDir, stockCodes[i], startYear, number)
dfi = readWSDIndexFile(baseDir, stockCodes[i], startYear, number)
allDF = pd.concat([df, dfi], axis=1)
print ("Factors Shape:", np.shape(df), np.shape(dfi))

# normalize
scaler = preprocessing.MinMaxScaler()
input_data = scaler.fit_transform(allDF)
shape = np.shape(input_data)
print ("input data shape: ", shape) # 489 day * 50 factors
sample_num = shape[0]
dim_num = shape[1]
ratio = 0.9
threshold = (int)(len(input_data) * ratio)
x_train, x_test = input_data[:threshold], input_data[threshold:]

############# AutoEncoder Model ##########################
encoding_dim = 30   # reduce dimensionality
batch_size = 20
epochs = 350
input_factors = Input(shape=(dim_num,))
# "encoded" is the encoded representation of the input
encoded = Dense(encoding_dim, activation='relu')(input_factors)
# "decoded" is the lossy reconstruction of the input
decoded = Dense(dim_num, activation='sigmoid')(encoded)
# this model maps an input to its reconstruction
autoencoder = Model(input_factors, decoded)

# compile
#autoencoder.compile(optimizer='rmsprop', loss='categorical_crossentropy')
#autoencoder.compile(optimizer='adadelta', loss='binary_crossentropy')
#autoencoder.compile(optimizer='rmsprop', loss='binary_crossentropy')
autoencoder.compile(loss='mean_squared_error', optimizer='adam')
# training
autoencoder.fit(x_train, x_train,
                epochs=epochs,
                batch_size=batch_size,
                shuffle=True,
                validation_data=(x_test, x_test))#, callbacks=[TensorBoard(log_dir='/home/dclab/tmp/feature_extract/')])


../000300.SH/000300.SH.csv ===============
Factors Shape: (727, 10) (727, 38)
input data shape:  (727, 48)
Train on 654 samples, validate on 73 samples
Epoch 1/350
654/654 [==============================] - 0s - loss: 0.0841 - val_loss: 0.0331
Epoch 2/350
654/654 [==============================] - 0s - loss: 0.0543 - val_loss: 0.0292
Epoch 3/350
654/654 [==============================] - 0s - loss: 0.0345 - val_loss: 0.0268
Epoch 4/350
654/654 [==============================] - 0s - loss: 0.0240 - val_loss: 0.0239
Epoch 5/350
654/654 [==============================] - 0s - loss: 0.0184 - val_loss: 0.0227
Epoch 6/350
654/654 [==============================] - 0s - loss: 0.0158 - val_loss: 0.0222
Epoch 7/350
654/654 [==============================] - 0s - loss: 0.0143 - val_loss: 0.0213
Epoch 8/350
654/654 [==============================] - 0s - loss: 0.0132 - val_loss: 0.0208
Epoch 9/350
654/654 [==============================] - 0s - loss: 0.0122 - val_loss: 0.0203
Epoch 10/350
654/654

654/654 [==============================] - 0s - loss: 0.0016 - val_loss: 0.0092
Epoch 88/350
654/654 [==============================] - 0s - loss: 0.0015 - val_loss: 0.0090
Epoch 89/350
654/654 [==============================] - 0s - loss: 0.0015 - val_loss: 0.0086
Epoch 90/350
654/654 [==============================] - 0s - loss: 0.0015 - val_loss: 0.0091
Epoch 91/350
654/654 [==============================] - 0s - loss: 0.0015 - val_loss: 0.0082
Epoch 92/350
654/654 [==============================] - 0s - loss: 0.0015 - val_loss: 0.0084
Epoch 93/350
654/654 [==============================] - 0s - loss: 0.0015 - val_loss: 0.0087
Epoch 94/350
654/654 [==============================] - 0s - loss: 0.0014 - val_loss: 0.0078
Epoch 95/350
654/654 [==============================] - 0s - loss: 0.0014 - val_loss: 0.0080
Epoch 96/350
654/654 [==============================] - 0s - loss: 0.0014 - val_loss: 0.0079
Epoch 97/350
654/654 [==============================] - 0s - loss: 0.0014 - val_los

654/654 [==============================] - 0s - loss: 7.7999e-04 - val_loss: 0.0059
Epoch 173/350
654/654 [==============================] - 0s - loss: 7.7017e-04 - val_loss: 0.0062
Epoch 174/350
654/654 [==============================] - 0s - loss: 7.6570e-04 - val_loss: 0.0062
Epoch 175/350
654/654 [==============================] - 0s - loss: 7.7194e-04 - val_loss: 0.0061
Epoch 176/350
654/654 [==============================] - 0s - loss: 7.6057e-04 - val_loss: 0.0060
Epoch 177/350
654/654 [==============================] - 0s - loss: 7.6446e-04 - val_loss: 0.0065
Epoch 178/350
654/654 [==============================] - 0s - loss: 7.4837e-04 - val_loss: 0.0061
Epoch 179/350
654/654 [==============================] - 0s - loss: 7.4457e-04 - val_loss: 0.0061
Epoch 180/350
654/654 [==============================] - 0s - loss: 7.4627e-04 - val_loss: 0.0062
Epoch 181/350
654/654 [==============================] - 0s - loss: 7.4399e-04 - val_loss: 0.0058
Epoch 182/350
654/654 [===========

654/654 [==============================] - 0s - loss: 5.6080e-04 - val_loss: 0.0049
Epoch 256/350
654/654 [==============================] - 0s - loss: 5.6416e-04 - val_loss: 0.0055
Epoch 257/350
654/654 [==============================] - 0s - loss: 5.6331e-04 - val_loss: 0.0048
Epoch 258/350
654/654 [==============================] - 0s - loss: 5.6423e-04 - val_loss: 0.0053
Epoch 259/350
654/654 [==============================] - 0s - loss: 5.6146e-04 - val_loss: 0.0050
Epoch 260/350
654/654 [==============================] - 0s - loss: 5.5700e-04 - val_loss: 0.0051
Epoch 261/350
654/654 [==============================] - 0s - loss: 5.5387e-04 - val_loss: 0.0049
Epoch 262/350
654/654 [==============================] - 0s - loss: 5.5858e-04 - val_loss: 0.0053
Epoch 263/350
654/654 [==============================] - 0s - loss: 5.5610e-04 - val_loss: 0.0052
Epoch 264/350
654/654 [==============================] - 0s - loss: 5.5148e-04 - val_loss: 0.0049
Epoch 265/350
654/654 [===========

654/654 [==============================] - 0s - loss: 4.7209e-04 - val_loss: 0.0049
Epoch 339/350
654/654 [==============================] - 0s - loss: 4.7505e-04 - val_loss: 0.0047
Epoch 340/350
654/654 [==============================] - 0s - loss: 4.7889e-04 - val_loss: 0.0046
Epoch 341/350
654/654 [==============================] - 0s - loss: 4.6945e-04 - val_loss: 0.0047
Epoch 342/350
654/654 [==============================] - 0s - loss: 4.6766e-04 - val_loss: 0.0049
Epoch 343/350
654/654 [==============================] - 0s - loss: 4.6929e-04 - val_loss: 0.0046
Epoch 344/350
654/654 [==============================] - 0s - loss: 4.6723e-04 - val_loss: 0.0046
Epoch 345/350
654/654 [==============================] - 0s - loss: 4.6588e-04 - val_loss: 0.0046
Epoch 346/350
654/654 [==============================] - 0s - loss: 4.6788e-04 - val_loss: 0.0050
Epoch 347/350
654/654 [==============================] - 0s - loss: 4.6399e-04 - val_loss: 0.0045
Epoch 348/350
654/654 [===========

In [ ]:
#optimizer='rmsprop', loss='binary_crossentropy',  encoding_dim = 20,  loss: 0.4696 - val_loss: 0.6308
#optimizer='adadelta', loss='binary_crossentropy' loss: 0.4767 - val_loss: 0.6546
#optimizer='rmsprop', encoding_dim = 40, loss: 0.4782 - val_loss: 0.6248
# encoding_dim = 8, loss: 0.4850 - val_loss: 0.6339
# encoding_dim = 18, loss: 0.4795 - val_loss: 0.6230
# encoding_dim = 30, loss: 0.4787 - val_loss: 0.6227
#loss='mean_squared_error', optimizer='adam', loss: 4.6516e-04 - val_loss: 0.0044